In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
import numpy as np
from retry_requests import retry

In [2]:
ort = (47.559, 7.587) #Basel
lat = ort[0]
long = ort[1]

In [41]:
weather_codes_en = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    45: "Fog",
    48: "Depositing rime fog",
    51: "Drizzle: Light",
    53: "Drizzle: Moderate",
    55: "Drizzle: Dense intensity",
    56: "Freezing Drizzle: Light",
    57: "Freezing Drizzle: Dense intensity",
    61: "Rain: Slight intensity",
    63: "Rain: Moderate intensity",
    65: "Rain: Heavy intensity",
    66: "Freezing Rain: Light intensity",
    67: "Freezing Rain: Heavy intensity",
    71: "Snow fall: Slight intensity",
    73: "Snow fall: Moderate intensity",
    75: "Snow fall: Heavy intensity",
    77: "Snow grains",
    80: "Rain showers: Slight intensity",
    81: "Rain showers: Moderate intensity",
    82: "Rain showers: Violent intensity",
    85: "Snow showers: Slight intensity",
    86: "Snow showers: Heavy intensity",
    95: "Thunderstorm: Slight or moderate",
    96: "Thunderstorm with slight hail",
    99: "Thunderstorm with heavy hail"
}

weather_codes_de = {
    0: "Klarer Himmel",
    1: "Teilweise klar",
    2: "Teilweise bewölkt",
    3: "Bedeckt",
    45: "Nebel",
    48: "Bildung von Reifnebel",
    51: "Nieselregen: Leicht",
    53: "Nieselregen: Mäßig",
    55: "Nieselregen: Stark",
    56: "Gefrierender Nieselregen: Leicht",
    57: "Gefrierender Nieselregen: Stark",
    61: "Regen: Leichte Intensität",
    63: "Regen: Mäßige Intensität",
    65: "Regen: Starke Intensität",
    66: "Gefrierender Regen: Leichte Intensität",
    67: "Gefrierender Regen: Starke Intensität",
    71: "Schneefall: Leichte Intensität",
    73: "Schneefall: Mäßige Intensität",
    75: "Schneefall: Starke Intensität",
    77: "Schneekörner",
    80: "Regenschauer: Leichte Intensität",
    81: "Regenschauer: Mäßige Intensität",
    82: "Regenschauer: Starke Intensität",
    85: "Schneeschauer: Leichte Intensität",
    86: "Schneeschauer: Starke Intensität",
    95: "Gewitter: Leicht oder mäßig",
    96: "Gewitter mit leichtem Hagel",
    99: "Gewitter mit schwerem Hagel"
}

In [42]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": lat,
	"longitude": long,
	"current": ["cloud_cover"],
	"hourly": ["weather_code", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "visibility"],
	"daily": ["weather_code", "sunrise", "sunset"],
	"timezone": "Europe/Berlin",
	"forecast_days": 3,
	"models": "best_match"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 47.560001373291016°N 7.5799994468688965°E
Elevation 261.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s


### Current

In [43]:
# Current values. The order of variables needs to be the same as requested.
current = response.Current()
current_cloud_cover = current.Variables(0).Value()

print(f"Current cloud_cover {current_cloud_cover}")

Current cloud_cover 64.0


### Hourly

In [56]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_weather_code = hourly.Variables(0).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(1).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(2).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(3).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(4).ValuesAsNumpy()
hourly_visibility = hourly.Variables(5).ValuesAsNumpy()
hourly_weather = np.vectorize(weather_codes_en.get)(hourly_weather_code)

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["weather_code"] = hourly_weather_code
hourly_data["weather"] = hourly_weather
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["visibility"] = hourly_visibility

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe

,date,weather_code,weather,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility
0,2024-03-18 23:00:00+00:00,3.0,Overcast,75.0,62.0,30.0,0.0,24140.0
1,2024-03-19 00:00:00+00:00,3.0,Overcast,92.0,86.0,49.0,3.0,24140.0
2,2024-03-19 01:00:00+00:00,3.0,Overcast,88.0,74.0,41.0,12.0,24140.0
3,2024-03-19 02:00:00+00:00,3.0,Overcast,87.0,74.0,19.0,41.0,24140.0
4,2024-03-19 03:00:00+00:00,45.0,Fog,91.0,76.0,37.0,36.0,24140.0
...,...,...,...,...,...,...,...,...
67,2024-03-21 18:00:00+00:00,0.0,Clear sky,1.0,0.0,1.0,0.0,24140.0
68,2024-03-21 19:00:00+00:00,1.0,Mainly clear,6.0,0.0,0.0,6.0,24140.0
69,2024-03-21 20:00:00+00:00,0.0,Clear sky,4.0,3.0,0.0,1.0,24140.0
70,2024-03-21 21:00:00+00:00,3.0,Overcast,96.0,9.0,4.0,94.0,24140.0


### Daily

In [57]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_sunrise = daily.Variables(1).ValuesAsNumpy()
daily_sunset = daily.Variables(2).ValuesAsNumpy()
daily_weather = np.vectorize(weather_codes_en.get)(daily_weather_code)

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["weather"] = daily_weather
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe

,date,weather_code,weather,sunrise,sunset
0,2024-03-18 23:00:00+00:00,45.0,Fog,0,0
1,2024-03-19 23:00:00+00:00,45.0,Fog,0,0
2,2024-03-20 23:00:00+00:00,80.0,Rain showers: Slight intensity,0,0


In [3]:
import openweather_api_func as ow

In [4]:
ow.openweather_hour(lat, long)

,date,weather_code,weather,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility
0,2024-03-19 00:00:00+00:00,3.0,Overcast,92.0,86.0,49.0,3.0,24140.0
1,2024-03-19 01:00:00+00:00,3.0,Overcast,88.0,74.0,41.0,12.0,24140.0
2,2024-03-19 02:00:00+00:00,3.0,Overcast,87.0,74.0,19.0,41.0,24140.0
3,2024-03-19 03:00:00+00:00,45.0,Fog,91.0,76.0,37.0,36.0,24140.0
4,2024-03-19 04:00:00+00:00,3.0,Overcast,90.0,83.0,30.0,12.0,24140.0
...,...,...,...,...,...,...,...,...
67,2024-03-21 19:00:00+00:00,2.0,Partly cloudy,56.0,39.0,0.0,24.0,24140.0
68,2024-03-21 20:00:00+00:00,3.0,Overcast,96.0,17.0,32.0,93.0,24140.0
69,2024-03-21 21:00:00+00:00,3.0,Overcast,99.0,46.0,22.0,97.0,24140.0
70,2024-03-21 22:00:00+00:00,3.0,Overcast,93.0,43.0,25.0,81.0,24140.0


In [4]:
ow.openweather_current(lat, long)

94.0